In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import plotly.graph_objs as go

# 1. Generar nube 3D con estructura planar
np.random.seed(42)
n = 100
x = np.random.normal(0, 2.0, size=n)
y = np.random.normal(0, 1.0, size=n)
z = np.random.normal(0, 0.1, size=n)
X = np.column_stack([x, y, z])

# Rotar aleatoriamente
theta = np.deg2rad(30)
phi = np.deg2rad(20)
Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
               [np.sin(theta),  np.cos(theta), 0],
               [0,              0,             1]])
Ry = np.array([[np.cos(phi), 0, np.sin(phi)],
               [0,           1, 0],
               [-np.sin(phi), 0, np.cos(phi)]])
X = X @ Rz.T @ Ry.T

# 2. PCA
pca = PCA(n_components=3)
pca.fit(X)
components = pca.components_
mean = pca.mean_
explained_var = pca.explained_variance_ratio_

# 3. PC1 y PC2 como líneas desde el centroide
vec_scale = 2
vector_lines = []
colors = ['crimson', 'seagreen']
names = ['PC1', 'PC2']
for i in range(2):
    v = components[i]
    vector_lines.append(go.Scatter3d(
        x=[mean[0] - vec_scale*v[0], mean[0] + vec_scale*v[0]],
        y=[mean[1] - vec_scale*v[1], mean[1] + vec_scale*v[1]],
        z=[mean[2] - vec_scale*v[2], mean[2] + vec_scale*v[2]],
        mode='lines',
        line=dict(color=colors[i], width=5),
        name=names[i]
    ))

# 4. Proyección sobre el plano PC1–PC2
PC1, PC2 = components[0], components[1]
PC_plane = np.vstack([PC1, PC2]).T
X_centered = X - mean
X_proj = X_centered @ PC_plane @ PC_plane.T + mean

# 5. Puntos originales y proyectados
scatter_orig = go.Scatter3d(
    x=X[:,0], y=X[:,1], z=X[:,2],
    mode='markers',
    marker=dict(size=3, color='steelblue'),
    name='Datos originales'
)

scatter_proj = go.Scatter3d(
    x=X_proj[:,0], y=X_proj[:,1], z=X_proj[:,2],
    mode='markers',
    marker=dict(size=3, color='darkorange'),
    name='Proyecciones'
)

# 6. Líneas desde cada punto a su proyección
lines = []
for i in range(n):
    lines.append(go.Scatter3d(
        x=[X[i,0], X_proj[i,0]],
        y=[X[i,1], X_proj[i,1]],
        z=[X[i,2], X_proj[i,2]],
        mode='lines',
        line=dict(color='gray', width=1),
        showlegend=False
    ))

# 7. Plano PC1-PC2
grid_size = 10
s = np.linspace(-3, 3, grid_size)
t = np.linspace(-3, 3, grid_size)
S, T = np.meshgrid(s, t)
plane_points = mean[:, None, None] + S[None,:,:]*PC1[:,None,None] + T[None,:,:]*PC2[:,None,None]
plane = go.Surface(
    x=plane_points[0], y=plane_points[1], z=plane_points[2],
    opacity=0.2, colorscale='Blues', showscale=False, name='Plano PC1-PC2'
)

# 8. Layout
layout = go.Layout(
    title=f"PCA 3D – Varianza: PC1 = {explained_var[0]*100:.1f}%, PC2 = {explained_var[1]*100:.1f}%",
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

# 9. Figura final
fig = go.Figure(
    data=[scatter_orig, scatter_proj, plane] + lines + vector_lines,
    layout=layout
)
fig.show()